In [10]:
import math

import torch
import torch.nn as nn

from diffusers.optimization import get_scheduler
from tqdm import tqdm

import pandas as pd
import numpy as np

import random

import os
device = "cuda"
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(0)


Random seed set as 0


In [ ]:
save_location = "/home/bhux/workplace/output/hptune/"
# save_location = "/media/bhux/alpha/xsd_mvp/test/"

# uniform, lg, gaussian

NUMERICAL = 12
CATEGORICAL = 0

INFILLING_TYPE = ''
NOISE_TYPE = ''

In [12]:
from utils import remove_outliers, norm, sample_local_gaussian, convert_categorical, unison_shuffled_copies

In [13]:
PATH = "./data/xtended_data_all.csv"
EMB_PATH = "./data/xtended_emb_all_deberta_pubchem.npy"

f = pd.read_csv(PATH)
drug_embeddings = np.load(EMB_PATH)
smiles = f['Drug'].values
vlists = {
    col: f[col].values for col in f.drop(labels=['Drug'], axis=1).columns[:NUMERICAL] 
}

inmask = remove_outliers([v for _,v in vlists.items()])
print(sum(inmask))
smiles = smiles[inmask]
vlists = {
    k: v[inmask] for k,v in vlists.items()
}

vlists = {
    k: norm(v) for k,v in vlists.items()
}

# for col in f.drop(labels=['Drug'], axis=1).columns[:NUMERICAL]:
#     vlists[col+"_cat"] = vlists[col]

nullmask = np.stack([
    np.isnan(v)==False for _,v in vlists.items()
    ], axis=-1)

dmss = []
for k,v in vlists.items():
    vlists[k], dms = sample_local_gaussian(v, numbins=15)
    dmss.append(dms)

for col in f.drop(labels=['Drug'], axis=1).columns[:NUMERICAL]:
    vlists[col+"_cat"] = convert_categorical(vlists[col], numbins=15)

# for col in f.drop(labels=['Drug'], axis=1).columns[NUMERICAL:]:
#     nan = np.isnan(vlists[col])
#     vlists[col] += 1
#     vlists[col][nan] = 0

# dmss = []
# for k,v in vlists.items():
#     dms = get_local_gaussian(v, numbins=50)
#     dmss.append(dms)

dataset = []
for i, gt in enumerate(zip(*[v for _,v in vlists.items()])):
    dataset.append({
        "sm": smiles[i],
        "ft": drug_embeddings[i],
        "ma": nullmask[i],
        "gt": np.array(gt),
        "od": np.array(gt[NUMERICAL:]),
    })
    # print(gt)
    # print(nullmask[i])
    # break

valCount = np.sum(nullmask, axis=0)*0.1
dataset, rcomb = unison_shuffled_copies(dataset, nullmask)
trdataset = []
valdataset = []
for c, d in zip(rcomb, dataset):
    inc = False
    for i, j in enumerate(list(c)):
        if j and valCount[i] > 0:
            valCount[i] -= 1
            inc = True
    if inc:
        valdataset.append(d)
    else:
        trdataset.append(d)

print(len(trdataset))
print(len(valdataset))
print(len(list(vlists.keys())))

28443
14
14
14
14
14
14
14
14
14
14
14
14
25603
2840
24


In [14]:
from torch.utils.data import Dataset

class GaucamolDataset(Dataset):
    def __init__(self, dataset) -> None:
        self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]
    
    def update(self, idx, delta):
        item = self.dataset[idx]["gt"]
        self.dataset[idx]["gt"] = item + delta

trainset = GaucamolDataset(trdataset)
valset = GaucamolDataset(valdataset)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=128, shuffle=True)
steps_per_epoch = len(trainset)
DMSS = dmss[:NUMERICAL]

In [15]:
def train(diffusion, ema, gamma, dataloader, optimizer, lr_scheduler, two_noise=False):
    diffusion.train()
    running_loss = 0
    global_step = 0
    for i, batch in enumerate(dataloader):
        ft = batch['ft'].to(device).float()
        gt = batch['gt'].to(device).float()
        od = batch['od'].to(device).long()
        mask = batch['ma'].to(device)
        bs = ft.shape[0]

        optimizer.zero_grad()
        loss_multi, loss_gauss = diffusion.mixed_loss(ft, gt, od, mask, DMSS)

        loss = loss_multi + loss_gauss
        
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        ema.update_params(gamma)
        gamma = ema.update_gamma(global_step)

        running_loss += loss.item()
        global_step += 1
    return running_loss/global_step

In [16]:
from sklearn.metrics import mean_squared_error
import csv
from utils import ohe_to_categories

def evaluate(e, ema, dataloader):
    ema.ema_model.eval()
    before_mse = 0
    running_mse = 0
    global_step = 0
    vals = {}
    device = 'cuda'
    ema.ema_model.to(device)
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            sm = batch['sm']
            mask = batch['ma'].repeat(1,2)
            ft = batch['ft'].to(device).float()
            gt = batch['gt'].to(device).float()
            od = batch['od'].to(device).long()
            bs = ft.shape[0]

            x_in, generated_ys = ema.ema_model.sample(ft, bs, od, DMSS, clip_sample=True)

            raw_mse = mean_squared_error(gt[mask].flatten().cpu(), x_in[mask].flatten().cpu())
            mse = mean_squared_error(gt[mask].flatten().cpu(), generated_ys[mask].flatten().cpu())

            for s, g in zip(sm, list(generated_ys.cpu().numpy())):
                vals[s] = g
            
            before_mse += raw_mse
            running_mse += mse
            global_step += 1

    with open(save_location+'{}_dict.csv'.format(e), 'w') as csv_file:  
        writer = csv.writer(csv_file)
        for key, value in vals.items():
            writer.writerow([key, value])

    return running_mse / global_step, before_mse / global_step
            

In [17]:
import logging
import sys
import optuna
from sdt import SDT
from diffusion import GaussianMultinomialDiffusion
torch.set_printoptions(profile="full")
from ema import EMA

num_epochs = 10

def objective(trial):
    lr = trial.suggest_float(
        "lr", 1e-5, 1e-2, log=True
    ) #0.0005

    wd = trial.suggest_float(
        "wd", 1e-5, 1e-2, log=True
    ) #1e-4

    warmup = trial.suggest_categorical(
        "warmup", [50, 100, 150, 200, 250, 300]
    ) #200

    gamma = trial.suggest_float(
        "gamma", 0.97, 0.999, log=True
    ) #0.994

    total_num_steps = (steps_per_epoch * num_epochs)

    model = SDT(
        time_dim = trial.suggest_categorical(
            "time_dim", [16, 32, 64]
        ), #64,
        cond_size = 768,
        patch_size = trial.suggest_categorical(
            "patch_size", [8, 16, 32, 64]
        ), #16
        y_dim = NUMERICAL+15*(NUMERICAL+CATEGORICAL),
        dim = 768,
        depth = trial.suggest_int(
            "depth", 4, 12
        ), #8,
        heads = trial.suggest_int(
            "heads", 4, 12
        ), #8,
        mlp_dim = trial.suggest_categorical(
            "mlp_dim", [256, 512, 768, 1024]
        ), #768,
        dropout = 0.1,
        emb_dropout = trial.suggest_float(
            "emb_dropout", 0, 0.2
        ), #0.1,
        num_classes = 15,
    )
    model.to(device)

    total_params = sum(p.numel() for p in model.parameters())
    print(f"Number of parameters: {total_params}")

    diffusion = GaussianMultinomialDiffusion(
        num_classes = np.array([15 for _ in range(NUMERICAL+CATEGORICAL)]),
        num_numerical_features = NUMERICAL,
        denoise_fn = model,
        device = device,
    )
    diffusion.to(device)

    ema = EMA(diffusion, gamma, total_num_steps)

    optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=lr,
            weight_decay=wd,
        )

    lr_scheduler = get_scheduler(
            trial.suggest_categorical(
                "schedule", 
                [
                    "cosine", 
                    "linear", 
                    "cosine_with_restarts", 
                    "constant", 
                    "constant_with_warmup",
                ]
            ), #"cosine",
            optimizer=optimizer,
            num_warmup_steps=warmup,
            num_training_steps=total_num_steps,
        )
    
    l = ""
    loss = 0
    for e in range(num_epochs):
        loss = train(diffusion, ema, gamma, trainloader, optimizer, lr_scheduler)

    mse, bmse = evaluate(e, ema, valloader)
    return mse / bmse


In [18]:
import pickle
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = save_location+"2024-10-28-optimize-ximagand"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

study = optuna.create_study(study_name=study_name, storage=storage_name, direction="minimize", load_if_exists=True)
study.optimize(objective, n_trials=1000)

trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

with open(save_location+"2024-10-28-sampler.pkl", "wb") as fout:
    pickle.dump(study.sampler, fout)

[I 2024-10-28 12:40:50,246] A new study created in RDB with name: /home/bhux/workplace/output/hptune/2024-10-28-optimize-ximagand


A new study created in RDB with name: /home/bhux/workplace/output/hptune/2024-10-28-optimize-ximagand
Number of parameters: 29971392
torch.Size([180])


[I 2024-10-28 12:42:25,755] Trial 0 finished with value: 0.7338032230201502 and parameters: {'lr': 0.005661939082123917, 'wd': 3.844047186212455e-05, 'warmup': 150, 'gamma': 0.9969809598397036, 'time_dim': 16, 'patch_size': 64, 'depth': 6, 'heads': 11, 'mlp_dim': 1024, 'emb_dropout': 0.15517643554336202, 'schedule': 'linear'}. Best is trial 0 with value: 0.7338032230201502.


Trial 0 finished with value: 0.7338032230201502 and parameters: {'lr': 0.005661939082123917, 'wd': 3.844047186212455e-05, 'warmup': 150, 'gamma': 0.9969809598397036, 'time_dim': 16, 'patch_size': 64, 'depth': 6, 'heads': 11, 'mlp_dim': 1024, 'emb_dropout': 0.15517643554336202, 'schedule': 'linear'}. Best is trial 0 with value: 0.7338032230201502.
Number of parameters: 13231744
torch.Size([180])


[I 2024-10-28 12:43:33,058] Trial 1 finished with value: 0.7491792282284524 and parameters: {'lr': 0.0010003508350898316, 'wd': 0.0031310129043516394, 'warmup': 250, 'gamma': 0.9892668983404174, 'time_dim': 16, 'patch_size': 32, 'depth': 5, 'heads': 5, 'mlp_dim': 256, 'emb_dropout': 0.18459599241966998, 'schedule': 'constant_with_warmup'}. Best is trial 0 with value: 0.7338032230201502.


...

Trial 997 finished with value: 0.6963335207608533 and parameters: {'lr': 0.00838318061368901, 'wd': 0.00018551522595627585, 'warmup': 250, 'gamma': 0.9967091443915953, 'time_dim': 32, 'patch_size': 64, 'depth': 6, 'heads': 7, 'mlp_dim': 768, 'emb_dropout': 0.07829810283664607, 'schedule': 'constant_with_warmup'}. Best is trial 598 with value: 0.49804964861220785.
Number of parameters: 22941184
torch.Size([180])


[I 2024-10-29 21:11:24,813] Trial 998 finished with value: 0.701618436050434 and parameters: {'lr': 0.007246816636377364, 'wd': 0.0014893913157145112, 'warmup': 300, 'gamma': 0.9983429016583535, 'time_dim': 32, 'patch_size': 32, 'depth': 6, 'heads': 7, 'mlp_dim': 768, 'emb_dropout': 0.07832708718652247, 'schedule': 'constant_with_warmup'}. Best is trial 598 with value: 0.49804964861220785.


Trial 998 finished with value: 0.701618436050434 and parameters: {'lr': 0.007246816636377364, 'wd': 0.0014893913157145112, 'warmup': 300, 'gamma': 0.9983429016583535, 'time_dim': 32, 'patch_size': 32, 'depth': 6, 'heads': 7, 'mlp_dim': 768, 'emb_dropout': 0.07832708718652247, 'schedule': 'constant_with_warmup'}. Best is trial 598 with value: 0.49804964861220785.
Number of parameters: 22941184
torch.Size([180])


[I 2024-10-29 21:13:01,578] Trial 999 finished with value: 0.7236711885250081 and parameters: {'lr': 0.005926061675366004, 'wd': 0.00023534831523452086, 'warmup': 300, 'gamma': 0.997903198396636, 'time_dim': 32, 'patch_size': 32, 'depth': 6, 'heads': 7, 'mlp_dim': 768, 'emb_dropout': 0.07554766372128713, 'schedule': 'cosine'}. Best is trial 598 with value: 0.49804964861220785.


Trial 999 finished with value: 0.7236711885250081 and parameters: {'lr': 0.005926061675366004, 'wd': 0.00023534831523452086, 'warmup': 300, 'gamma': 0.997903198396636, 'time_dim': 32, 'patch_size': 32, 'depth': 6, 'heads': 7, 'mlp_dim': 768, 'emb_dropout': 0.07554766372128713, 'schedule': 'cosine'}. Best is trial 598 with value: 0.49804964861220785.
Accuracy: 0.49804964861220785
Best hyperparameters: {'lr': 0.009980944985851876, 'wd': 0.0002561887904737375, 'warmup': 150, 'gamma': 0.9793712795623954, 'time_dim': 16, 'patch_size': 32, 'depth': 5, 'heads': 7, 'mlp_dim': 768, 'emb_dropout': 0.1423146531023044, 'schedule': 'constant'}


In [19]:
print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

Accuracy: 0.49804964861220785
Best hyperparameters: {'lr': 0.009980944985851876, 'wd': 0.0002561887904737375, 'warmup': 150, 'gamma': 0.9793712795623954, 'time_dim': 16, 'patch_size': 32, 'depth': 5, 'heads': 7, 'mlp_dim': 768, 'emb_dropout': 0.1423146531023044, 'schedule': 'constant'}
